<a href="https://colab.research.google.com/github/yeesem/Natural-Laguage-Processing/blob/main/Binary_Classifier_with_IMDB_Reviews_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install this package if running on your local machine
!pip install -q tensorflow-datasets

In [6]:
import tensorflow_datasets as tfds

# Load the IMDB Reviews dataset
imdb,info = tfds.load("imdb_reviews",with_info = True,as_supervised = True)

In [4]:
print(info)

tfds.core.DatasetInfo(
    name='imdb_reviews',
    full_name='imdb_reviews/plain_text/1.0.0',
    description="""
    Large Movie Review Dataset. This is a dataset for binary sentiment
    classification containing substantially more data than previous benchmark
    datasets. We provide a set of 25,000 highly polar movie reviews for training,
    and 25,000 for testing. There is additional unlabeled data for use as well.
    """,
    config_description="""
    Plain text
    """,
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    data_dir='/root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0',
    file_format=tfrecord,
    download_size=80.23 MiB,
    dataset_size=129.83 MiB,
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
        'text': Text(shape=(), dtype=string),
    }),
    supervised_keys=('text', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=25000, num_shards=1>,
    

In [7]:
# Print the contents of the dataset
print(imdb)

{'train': <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>, 'test': <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>, 'unsupervised': <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}


In [8]:
# Take 2 training examples and print its contents
for example in imdb['train'].take(2):
  print(example)

(<tf.Tensor: shape=(), dtype=string, numpy=b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.">, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on

In [21]:
import numpy as np

# Get the train and test sets
train_data,test_data = imdb['train'],imdb['test']

# Initialize sentences and labels lists
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# Loop over all training examples and save the sentences and labels
for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())

for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())

# Convert labels lists to numpy array
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [16]:
# Generate padded sequences

# Parameters
vocab_size = 10000
max_length = 120
embedding_dim = 16
trunc_type = 'post'
oov_tok = '<OOV>'

In [22]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Initialize the Tokenizer class
tokenizer = Tokenizer(num_words = vocab_size,oov_token = oov_tok)

# Generate the word index dictionary for the training sentences
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

# Generate and pad the training sequences
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen = max_length,truncating = trunc_type)

# Generate and pad the testing sequences
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen = max_length,truncating = trunc_type)

In [26]:
# Build and compile the Model
import tensorflow as tf

# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length = max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6,activation = 'relu'),
    tf.keras.layers.Dense(1,activation = 'sigmoid')
])

# Setup the training parameters
model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

# Print the model summary
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 120, 16)           160000    
                                                                 
 flatten_3 (Flatten)         (None, 1920)              0         
                                                                 
 dense_6 (Dense)             (None, 6)                 11526     
                                                                 
 dense_7 (Dense)             (None, 1)                 7         
                                                                 
Total params: 171533 (670.05 KB)
Trainable params: 171533 (670.05 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
# Train the model
num_epochs = 10

# Train the model
model.fit(
    padded,
    training_labels_final,
    epochs = num_epochs,
    validation_data = (testing_padded,testing_labels_final)
    )

Epoch 1/10
782/782 [==============================] - 7s 7ms/step - loss: 0.4877 - accuracy: 0.7485 - val_loss: 0.3844 - val_accuracy: 0.8271
Epoch 2/10
782/782 [==============================] - 7s 9ms/step - loss: 0.2421 - accuracy: 0.9053 - val_loss: 0.4204 - val_accuracy: 0.8165
Epoch 3/10
782/782 [==============================] - 6s 7ms/step - loss: 0.0984 - accuracy: 0.9733 - val_loss: 0.5087 - val_accuracy: 0.8093
Epoch 4/10
782/782 [==============================] - 6s 8ms/step - loss: 0.0248 - accuracy: 0.9970 - val_loss: 0.6230 - val_accuracy: 0.8048
Epoch 5/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0061 - accuracy: 0.9997 - val_loss: 0.6837 - val_accuracy: 0.8095
Epoch 6/10
782/782 [==============================] - 7s 9ms/step - loss: 0.0023 - accuracy: 0.9999 - val_loss: 0.7488 - val_accuracy: 0.8103
Epoch 7/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0013 - accuracy: 0.9999 - val_loss: 0.8079 - val_accuracy: 0.8087
Epoch 

In [30]:
# Get the embeddings layer from model
embedding_layer = model.layers[0]

# Get the weights of the embedding layer
embedding_weights = embedding_layer.get_weights()[0]

# Print the shape. (vocab_size,embedding_dim)
print(embedding_weights.shape)

(10000, 16)
